In [3]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv

from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma

import gradio as gr

load_dotenv()


In [4]:

yelp = pd.read_csv("Yelp_3k_Aspects_&_Emotions.csv")


In [3]:
import gradio as gr
import json
import re

# # Load the JSON data
# with open('yelp3k.json', 'r') as f:
#     reviews = json.load(f)
json_path = "Yelp3k.json"

# Function to highlight aspect words in the review text
# Ensure DataFrame has columns: text, aspect_word, emotions, sentiment
records = yelp.to_dict(orient="records")
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(records, f, ensure_ascii=False, indent=2)

# Step 2: Load JSON
with open(json_path, encoding="utf-8") as f:
    reviews = json.load(f)
def highlight_aspects(text, aspects):
    aspects_sorted = sorted(aspects, key=len, reverse=True)
    aspects_regex = [re.escape(asp) for asp in aspects_sorted]
    pattern = r'\b(?:' + '|'.join(aspects_regex) + r')\b'
    def replace_match(match):
        return f'<span style="font-weight: bold; background-color: yellow;">{match.group()}</span>'
    highlighted = re.sub(pattern, replace_match, text, flags=re.IGNORECASE)
    return highlighted

# Function to get the current review's data
def get_review(index):
    review = reviews[index]
    text = review['text']
    aspects = [asp.strip() for asp in review['aspect_word'].split(',')]
    stars = review['stars']
    emotions = [emo.strip() for emo in review['emotion'].split(',')]
    
    # Highlight aspect words in the text
    highlighted_text = highlight_aspects(text, aspects)
    
    # Format aspects and emotions as HTML lists
    aspects_html = "<ul>" + "".join(f"<li>{asp}</li>" for asp in aspects) + "</ul>"
    emotions_html = "<ul>" + "".join(f"<li>{emo}</li>" for emo in emotions) + "</ul>"
    
    # Format stars and review info
    stars_html = f"Stars: {stars}"
    review_info_text = f"Review {index+1} of {len(reviews)}"
    
    return highlighted_text, stars_html, aspects_html, emotions_html, review_info_text

# Functions for navigation
def previous(index):
    return max(index - 1, 0)

def next(index):
    return min(index + 1, len(reviews) - 1)

# Create the Gradio interface
with gr.Blocks() as demo:
    # State to keep track of the current review index
    current_index = gr.State(0)
    
    # Header
    gr.Markdown("# Yelp Review Demonstration for Aspect and Emotion Extracted")
    
    # Main content row with text and stars
    with gr.Row():
        with gr.Column(scale=10):
            text_display = gr.HTML(label="Review Text")
        with gr.Column(scale=2):
            stars_display = gr.HTML(label="Stars")
    
    # Full-width sections for aspects and emotions
    aspects_display = gr.HTML(label="Aspect Words")
    emotions_display = gr.HTML(label="Emotions")
    
    # Review info display
    review_info = gr.Textbox(label="Review Info", interactive=False)
    
    # Navigation buttons
    prev_button = gr.Button("Previous")
    next_button = gr.Button("Next")
    
    # Load initial review
    demo.load(get_review, inputs=current_index, outputs=[text_display, stars_display, aspects_display, emotions_display, review_info])
    
    # Event handlers for navigation
    prev_button.click(previous, inputs=current_index, outputs=current_index).then(
        get_review, inputs=current_index, outputs=[text_display, stars_display, aspects_display, emotions_display, review_info]
    )
    
    next_button.click(next, inputs=current_index, outputs=current_index).then(
        get_review, inputs=current_index, outputs=[text_display, stars_display, aspects_display, emotions_display, review_info]
    )

# Launch the dashboard
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://425deeb59f4f31508b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import json
import re
import html

# Load the JSON data
json_path = "Yelp3k.json"

# Function to highlight aspect words in the review text
# Ensure DataFrame has columns: text, aspect_word, emotions, sentiment
records = yelp.to_dict(orient="records")
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(records, f, ensure_ascii=False, indent=2)

# Function to highlight aspect words in the review text
def highlight_aspects(text, aspects):
    aspects_sorted = sorted(aspects, key=len, reverse=True)
    aspects_regex = [re.escape(asp) for asp in aspects_sorted]
    pattern = r'\b(?:' + '|'.join(aspects_regex) + r')\b'
    def replace_match(match):
        text_eligent=(f'<div class="inline-item">{match.group()}</div>') 
        return text_eligent #(f'<span class="inline-item">{match.group()}</span>') 
   # f'<span style="font-weight: bold; background-color: yellow;">{match.group()}</span>'
    highlighted = re.sub(pattern, replace_match, text, flags=re.IGNORECASE)
    return highlighted

# Function to get the current review's data
def get_review(index):
    review = reviews[index]
    text = review['text']
    aspects = [asp.strip() for asp in review['aspect_word'].split(',')]
    stars = review['stars']
    emotions = [emo.strip() for emo in review['emotion'].split(',')]
    
    # Highlight aspect words in the text
    highlighted_text = highlight_aspects(text, aspects)
    
    # Format aspects as a grid of HTML
    aspects_html = '<div class="cell-grid">' + "".join(f'<div class="cell-item">{html.escape(asp)}</div>' for asp in aspects) + '</div>'
    emotion_html = '<div class="cell-grid">' + "".join(f'<div class="cell-item">{html.escape(emo)}</div>' for emo in emotions) + '</div>'
    
    # Format emotions as an HTML list
    emotions_html = "<ul>" + "".join(f"<li>{emo}</li>" for emo in emotions) + "</ul>"
    
    # Format stars and review info
    stars_html = f"Stars: {stars}"
    review_info_text = f"Review {index+1} of {len(reviews)}"
    
    return highlighted_text, stars_html, aspects_html, emotion_html, review_info_text

# Functions for navigation
def previous(index):
    return max(index - 1, 0)

def next(index):
    return min(index + 1, len(reviews) - 1)

# Create the Gradio interface
with gr.Blocks(css="""
.cell-grid {
  display: grid;
  grid-template-columns: repeat(8, 1fr); /* 6 columns, each taking 1/6 of the width (equivalent to 2/12) */
  gap: 2px; /* Space between grid items */
}

.cell-item {
  background-color: #f0f0f0; /* Light gray background */
  padding: 2px;
  border: 1px solid #ccc;
  text-align: center;
}
.inline-item {
  background-color: #f0f0f0; /* Light gray background */
  padding: 2px;
  border: 1px solid #ccc;
  text-align: center;
}
""") as demo:
    # State to keep track of the current review index
    current_index = gr.State(0)
    
    # Header
    gr.Markdown("# Yelp Review Demonstration for Aspect and Emotion Extracted")
    
    # Main content row with text and stars
    with gr.Row():
        with gr.Column(scale=11):
            text_display = gr.HTML(label="Review Text")
        with gr.Column(scale=1):
            stars_display = gr.HTML(label="Stars")
    
    # Full-width section for aspects (grid layout)
    aspects_display = gr.HTML( label="Aspect Words")
    
    # Full-width section for emotions (list layout)
    emotions_display = gr.HTML(label="Emotions")
    
    # Review info display
    review_info = gr.Textbox(label="Review Info", interactive=False)
    
    # Navigation buttons
    prev_button = gr.Button("Previous")
    next_button = gr.Button("Next")
    
    # Load initial review
    demo.load(get_review, inputs=current_index, outputs=[text_display, stars_display, aspects_display, emotions_display, review_info])
    
    # Event handlers for navigation
    prev_button.click(previous, inputs=current_index, outputs=current_index).then(
        get_review, inputs=current_index, outputs=[text_display, stars_display, aspects_display, emotions_display, review_info]
    )
    
    next_button.click(next, inputs=current_index, outputs=current_index).then(
        get_review, inputs=current_index, outputs=[text_display, stars_display, aspects_display, emotions_display, review_info]
    )

# Launch the dashboard
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://4525cada0370c27af3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "c:\Users\khan\PycharmProjects\llm-yelp-rec\.venv\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\khan\PycharmProjects\llm-yelp-rec\.venv\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\khan\PycharmProjects\llm-yelp-rec\.venv\Lib\site-packages\gradio\blocks.py", line 2047, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\khan\PycharmProjects\llm-yelp-rec\.venv\Lib\site-packages\gradio\blocks.py", line 1594, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\khan\PycharmProjects\llm-yelp-rec\.venv\Lib\s